In [ ]:

import sys
sys.path.append('..')
sys.path.append('../ehrshot')
import argparse
import json
import os
from typing import Any, Dict, List, Optional, Tuple, Union
import numpy as np
import collections
import pandas as pd
import pickle
from datetime import datetime
from loguru import logger
from utils import (
    LABELING_FUNCTION_2_PAPER_NAME,
    SHOT_STRATS,
    MODEL_2_INFO,
    get_labels_and_features, 
    process_chexpert_labels, 
    convert_multiclass_to_binary_labels,
    CHEXPERT_LABELS, 
    LR_PARAMS, 
    XGB_PARAMS, 
    RF_PARAMS,
    ProtoNetCLMBRClassifier, 
    get_patient_splits_by_idx
)


import femr
import femr.datasets
from femr.labelers import load_labeled_patients, LabeledPatients

import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
torch.manual_seed(42)
from tqdm import tqdm

In [ ]:
with open('../EHRSHOT_ASSETS/features/clmbr_features.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
time = data['labeling_time']

In [ ]:
data

In [ ]:
df_general_operation = pd.read_csv('../white_box_common_data/general_operation.csv')
df_general_operation

df_lab_test

def add_feature_index(df):
    common_index_list = []
    for i in tqdm(range(len(df))):
        prediction_time = df.loc[i, 'prediction_time']
        prediction_time = datetime.strptime(prediction_time, '%Y-%m-%d %H:%M:%S')
        patient_id = df.loc[i, 'patient_id']

        mask1 = data['labeling_time'] == prediction_time
        mask2 = data['patient_ids'] == patient_id

        common_index = np.where(mask1 & mask2)[0]
        assert len(common_index) == 1
        common_index = common_index[0]
        common_index_list.append(common_index)
    df['feature_index'] = common_index_list
    return df

def add_feature_index_v2(df):
    common_index_list = []
    for i in tqdm(range(len(df))):
        prediction_time = df.loc[i, 'admission_date']
        prediction_time = datetime.strptime(prediction_time, '%Y-%m-%d %H:%M:%S')
        patient_id = df.loc[i, 'patient_id']

        mask1 = data['labeling_time'] == prediction_time
        mask2 = data['patient_ids'] == patient_id

        common_index = np.where(mask1 & mask2)[0]
        assert len(common_index) == 1
        common_index = common_index[0]
        common_index_list.append(common_index)
    df['feature_index'] = common_index_list
    return df

In [ ]:
df_general_operation = pd.read_csv('../white_box_common_data/general_operation.csv')
df_general_operation_with_index = add_feature_index(df_general_operation)
df_general_operation_with_index.to_csv('../white_box_common_data/general_operation_with_index.csv', index=False)

In [ ]:
df_general_operation_v2 = pd.read_csv('../white_box_common_data/general_operation_v2.csv')
df_general_operation_with_index_v2 = add_feature_index_v2(df_general_operation_v2)
df_general_operation_with_index_v2.to_csv('../white_box_common_data/general_operation_with_index_v2.csv', index=False)

In [ ]:
df_general_operation_v2

In [ ]:
df_new_diagnose = pd.read_csv('../white_box_common_data/new_diagnose.csv')
df_new_diagnose_with_index = add_feature_index(df_new_diagnose)
df_new_diagnose_with_index.to_csv('../white_box_common_data/new_diagnose_with_index.csv', index=False)

In [ ]:
df_lab_test = pd.read_csv('../white_box_common_data/lab_test.csv')
df_lab_test_with_index = add_feature_index(df_lab_test)
df_lab_test_with_index.to_csv('../white_box_common_data/lab_test_with_index.csv', index=False)

In [ ]:
data.keys(), data['labeling_time'].shape, data['patient_ids'].shape, data['data_matrix'].shape

In [ ]:
idx_general_operation = df_general_operation_with_index['feature_index'].values

In [ ]:
feature_general_operation = data['data_matrix'][idx_general_operation]

In [ ]:
df_general_operation_with_index

In [ ]:
multi_task_dir = '../same_time_data/'
multi_task_name = ['general_operation_v1', '','new_diagnose', 'lab_test']
df_split = pd.read_csv('../EHRSHOT_ASSETS/splits/person_id_map.csv')

In [ ]:
np.unique(df_split)